# Activity: Build a random forest model

## **Introduction**


As you're learning, random forests are popular statistical learning algorithms. Some of their primary benefits include reducing variance, bias, and the chance of overfitting.

This activity is a continuation of the project you began modeling with decision trees for an airline. Here, you will train, tune, and evaluate a random forest model using data from spreadsheet of survey responses from 129,880 customers. It includes data points such as class, flight distance, and inflight entertainment. Your random forest model will be used to predict whether a customer will be satisfied with their flight experience.

**Note:** Because this lab uses a real dataset, this notebook first requires exploratory data analysis, data cleaning, and other manipulations to prepare it for modeling.

## **Step 1: Imports** 


Import relevant Python libraries and modules, including `numpy` and `pandas`libraries for data processing; the `pickle` package to save the model; and the `sklearn` library, containing:
- The module `ensemble`, which has the function `RandomForestClassifier`
- The module `model_selection`, which has the functions `train_test_split`, `PredefinedSplit`, and `GridSearchCV` 
- The module `metrics`, which has the functions `f1_score`, `precision_score`, `recall_score`, and `accuracy_score`


In [2]:
# Import `numpy`, `pandas`, `pickle`, and `sklearn`.
# Import the relevant functions from `sklearn.ensemble`, `sklearn.model_selection`, and `sklearn.metrics`.

### YOUR CODE HERE ###
# Import standard data processing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Import scikit-learn modules for creating, validating and evaluating random forests
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,\
ConfusionMatrixDisplay

from sklearn.ensemble import RandomForestClassifier


As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

In [2]:
# RUN THIS CELL TO IMPORT YOUR DATA. 

### YOUR CODE HERE ###

air_data = pd.read_csv("Invistico_Airline.csv")

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `read_csv()` function from the `pandas` library can be helpful here.
 
</details>

Now, you're ready to begin cleaning your data. 

## **Step 2: Data cleaning** 

To get a sense of the data, display the first 10 rows.

In [3]:
# Display first 10 rows.

### YOUR CODE HERE ###
air_data.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `head()` function from the `pandas` library can be helpful here.
 
</details>

Now, display the variable names and their data types. 

In [4]:
# Display variable names and types.

### YOUR CODE HERE ###
air_data.dtypes

satisfaction                          object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
dtype: obj

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

DataFrames have an attribute that outputs variable names and data types in one result.
 
</details>

**Question:** What do you observe about the differences in data types among the variables included in the data?

There are 22 variables in the data of which 4 are not numeric.

Next, to understand the size of the dataset, identify the number of rows and the number of columns.

In [5]:
# Identify the number of rows and the number of columns.

### YOUR CODE HERE ###
air_data.shape

(129880, 22)

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

There is a method in the `pandas` library that outputs the number of rows and the number of columns in one result.

</details>

Now, check for missing values in the rows of the data. Start with .isna() to get Booleans indicating whether each value in the data is missing. Then, use .any(axis=1) to get Booleans indicating whether there are any missing values along the columns in each row. Finally, use .sum() to get the number of rows that contain missing values.

In [6]:
# Get Booleans to find missing values in data.
# Get Booleans to find missing values along columns.
# Get the number of rows that contain missing values.

### YOUR CODE HERE ###
air_data.isna().any(axis=1).sum()

393

**Question:** How many rows of data are missing values?**

There are 393 rows with missing values in them.

Drop the rows with missing values. This is an important step in data cleaning, as it makes the data more useful for analysis and regression. Then, save the resulting pandas DataFrame in a variable named `air_data_subset`.

In [7]:
# Drop missing values.
# Save the DataFrame in variable `air_data_subset`.

### YOUR CODE HERE ###
air_data_subset = air_data.dropna(how='any')

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

The `dropna()` function is helpful here.
</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The axis parameter passed in to this function should be set to 0 (if you want to drop rows containing missing values) or 1 (if you want to drop columns containing missing values).
</details>

Next, display the first 10 rows to examine the data subset.

In [8]:
# Display the first 10 rows.

### YOUR CODE HERE ###
air_data_subset.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


Confirm that it does not contain any missing values.

In [9]:
# Count of missing values.

### YOUR CODE HERE ###
air_data_subset.isna().any(axis=1).sum()

0

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `.isna().sum()` to get the number of missing values for each variable.

</details>

Next, convert the categorical features to indicator (one-hot encoded) features. 

**Note:** The `drop_first` argument can be kept as default (`False`) during one-hot encoding for random forest models, so it does not need to be specified. Also, the target variable, `satisfaction`, does not need to be encoded and will be extracted in a later step.

In [10]:
# Convert categorical features to one-hot encoded features.

### YOUR CODE HERE ###
air_data_subset_dummies = pd.get_dummies(air_data_subset,drop_first=True)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `pd.get_dummies()` function to convert categorical variables to one-hot encoded variables.
</details>

**Question:** Why is it necessary to convert categorical data into dummy variables?**

Because the random forest model we will be using only work with numeric data.

Next, display the first 10 rows to review the `air_data_subset_dummies`. 

In [11]:
# Display the first 10 rows.

### YOUR CODE HERE ###
air_data_subset_dummies.head(10)

,Age,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,...,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction_satisfied,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus
0,65,265,0,0,0,2,2,4,2,3,...,5,3,2,0,0.0,1,0,1,1,0
1,47,2464,0,0,0,3,0,2,2,3,...,2,3,2,310,305.0,1,0,1,0,0
2,15,2138,0,0,0,3,2,0,2,2,...,4,4,2,0,0.0,1,0,1,1,0
3,60,623,0,0,0,3,3,4,3,1,...,4,1,3,0,0.0,1,0,1,1,0
4,70,354,0,0,0,3,4,3,4,2,...,4,2,5,0,0.0,1,0,1,1,0
5,30,1894,0,0,0,3,2,0,2,2,...,5,4,2,0,0.0,1,0,1,1,0
6,66,227,0,0,0,3,2,5,5,5,...,5,5,3,17,15.0,1,0,1,1,0
7,10,1812,0,0,0,3,2,0,2,2,...,5,4,2,0,0.0,1,0,1,1,0
8,56,73,0,0,0,3,5,3,5,4,...,5,4,4,0,0.0,1,0,1,0,0
9,22,1556,0,0,0,3,2,0,2,2,...,3,4,2,30,26.0,1,0,1,1,0


Then, check the variables of air_data_subset_dummies.

In [12]:
# Display variables.

### YOUR CODE HERE ###
air_data_subset_dummies.dtypes

Age                                    int64
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
satisfaction_satisfied                 uint8
Customer Type_disloyal Customer        uint8
Type of Travel_Personal Travel         uint8
Class_Eco                              uint8
Class_Eco 

**Question:** What changes do you observe after converting the string data to dummy variables?**

All the dummy variables are of type `uint8` which is just an 8 bit integer so its range of possible values is less than `int64`

## **Step 3: Model building** 

The first step to building your model is separating the labels (y) from the features (X).

In [13]:
# Separate the dataset into labels (y) and features (X).

### YOUR CODE HERE ###
X = air_data_subset_dummies.drop(columns=['satisfaction_satisfied'])
Y = air_data_subset_dummies['satisfaction_satisfied']

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Save the labels (the values in the `satisfaction` column) as `y`.

Save the features as `X`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

To obtain the features, drop the `satisfaction` column from the DataFrame.

</details>

Once separated, split the data into train, validate, and test sets. 

In [14]:
# Separate into train, validate, test sets.

### YOUR CODE HERE ###
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25,
                                                    stratify=Y, random_state=42)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2,
                                           stratify=y_train, random_state=42)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `train_test_split()` function twice to create train/validate/test sets, passing in `random_state` for reproducible results. 

</details>

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Split `X`, `y` to get `X_train`, `X_test`, `y_train`, `y_test`. Set the `test_size` argument to the proportion of data points you want to select for testing. 

Split `X_train`, `y_train` to get `X_tr`, `X_val`, `y_tr`, `y_val`. Set the `test_size` argument to the proportion of data points you want to select for validation. 

</details>

### Tune the model

Now, fit and tune a random forest model with separate validation set. Begin by determining a set of hyperparameters for tuning the model using GridSearchCV.


In [15]:
# Determine set of hyperparameters.

### YOUR CODE HERE ###
cv_params = {'max_depth': [2,3,4,5, None], 
             'min_samples_leaf': [1,2,3],
             'min_samples_split': [2,3,4],
             'max_features': [2,3,4],
             'n_estimators': [75, 100, 125, 150]
             }  

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Create a dictionary `cv_params` that maps each hyperparameter name to a list of values. The GridSearch you conduct will set the hyperparameter to each possible value, as specified, and determine which value is optimal.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The main hyperparameters here include `'n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'max_features', and 'max_samples'`. These will be the keys in the dictionary `cv_params`.

</details>

Next, create a list of split indices.

In [16]:
# Create list of split indices.

### YOUR CODE HERE ###
split_index = [0 if x in X_val.index else -1 for x in X_train.index]

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use list comprehension, iterating over the indices of `X_train`. The list can consists of 0s to indicate data points that should be treated as validation data and -1s to indicate data points that should be treated as training data.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Use `PredfinedSplit()`, passing in `split_index`, saving the output as `custom_split`. This will serve as a custom split that will identify which data points from the train set should be treated as validation data during GridSearch.

</details>

Now, instantiate your model.

In [17]:
# Instantiate model.

### YOUR CODE HERE ### 
rf = RandomForestClassifier(random_state=0)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results. This will help you instantiate a random forest model, `rf`.

</details>

Next, use GridSearchCV to search over the specified parameters.

In [18]:
# Search over specified parameters.

### YOUR CODE HERE ### 
custom_split = PredefinedSplit(split_index)

scoring = {'accuracy', 'precision', 'recall', 'f1'}

rf_val = GridSearchCV(rf, cv_params, scoring=scoring, cv=custom_split, refit='f1')

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `GridSearchCV()`, passing in `rf` and `cv_params` and specifying `cv` as `custom_split`. Additional arguments that you can specify include: `refit='f1', n_jobs = -1, verbose = 1`. 

</details>

Now, fit your model.

In [19]:
%%time
# Fit the model.

### YOUR CODE HERE ###
#balls cum pee
rf_val.fit(X_train, y_train)

CPU times: user 31min 51s, sys: 6.14 s, total: 31min 57s
Wall time: 31min 57s


GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0, -1, ..., -1,  0])),
             error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight...
                                              oob_score=False, random_state=0,
                                              verbose=0, warm

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train the GridSearchCV model on `X_train` and `y_train`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Add the magic function `%%time` to keep track of the amount of time it takes to fit the model and display this information once execution has completed. Remember that this code must be the first line in the cell.

</details>

Finally, obtain the optimal parameters.

In [20]:
# Obtain optimal parameters.

### YOUR CODE HERE ###
rf_val.best_params_

{'max_depth': None,
 'max_features': 4,
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 100}

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `best_params_` attribute to obtain the optimal values for the hyperparameters from the GridSearchCV model.

</details>

## **Step 4: Results and evaluation** 

Use the selected model to predict on your test data. Use the optimal parameters found via GridSearchCV.

In [21]:
# Use optimal parameters on GridSearchCV.

### YOUR CODE HERE ###
rf_opt = RandomForestClassifier(n_estimators=100, min_samples_split=3,
                               min_samples_leaf=1, max_depth=None, max_features=4)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results and passing in the optimal hyperparameters found in the previous step. To distinguish this from the previous random forest model, consider naming this variable `rf_opt`.

</details>

Once again, fit the optimal model.

In [22]:
# Fit the optimal model.

### YOUR CODE HERE ###
rf_opt.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=4,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train `rf_opt` on `X_train` and `y_train`.

</details>

And predict on the test set using the optimal model.

In [23]:
# Predict on test set.

### YOUR CODE HERE ###
y_pred = rf_opt.predict(X_test)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `predict()` function to make predictions on `X_test` using `rf_opt`. Save the predictions now (for example, as `y_pred`), to use them later for comparing to the true labels. 

</details>

### Obtain performance scores

First, get your precision score.

In [24]:
# Get precision score.

### YOUR CODE HERE ###
print("Precision Score:",round(precision_score(y_test, y_pred),4))

Precision Score: 0.9641


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `precision_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Then, collect the recall score.

In [25]:
# Get recall score.

### YOUR CODE HERE ###
print("Recall Score:",round(recall_score(y_test, y_pred),4))

Recall Score: 0.9493


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `recall_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Next, obtain your accuracy score.

In [26]:
# Get accuracy score.

### YOUR CODE HERE ###
print("Accuracy Score:",round(accuracy_score(y_test, y_pred),4))

Accuracy Score: 0.9529


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `accuracy_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Finally, collect your F1-score.

In [27]:
# Get F1 score.

### YOUR CODE HERE ###
print("F1 Score:",round(f1_score(y_test, y_pred),4))

F1 Score: 0.9566


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `f1_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

**Question:** How is the F1-score calculated?

The following equation is used to calculate the F1 score
$$
F1 = 2\frac{(precision \times recall)}{(precision + recall)}
$$

**Question:** What are the pros and cons of performing the model selection using test data instead of a separate validation dataset?

Pros:
- The code become shorter
- No need to train data 2 times, (train and validation sets)

Cons:
- The model will be more prone to overfitting becuase we are using the testing data to validate the model.
- The overall evaluation of the model will be biased.



### Evaluate the model

Now that you have results, evaluate the model. 

**Question:** What are the four basic parameters for evaluating the performance of a classification model?

The four basic parameters for evaluating the performance of a model are:

1) Detecting True Positves

2) Detecting True Negatives

3) Avoiding False Negatives

4) Avoiding False Positives

**Question:**  What do the four scores demonstrate about your model, and how do you calculate them?

Accuracy measures the overall correct classification of the model for both true positves and true negatives.

$$
Accuracy = \frac{\text{TP} + \text{TN}}{\text{FP} + \text{FN} + \text{TP} + \text{TN}}
$$

Precision measures the proportion of positive identifications that were actually correct

$$
Precision = \frac{\text{TP}}{\text{TP} + \text{FP}}
$$

Recall measures the proportion of *actual* positives that were identified correctly

$$
Recall = \frac{\text{TP}}{\text{TP} + \text{FN}}
$$

F1 is the harmonic mean of the Precision and Recall scores

$$
F1 = 2\frac{(precision \times recall)}{(precision + recall)}
$$

Calculate the scores: precision score, recall score, accuracy score, F1 score.

In [28]:
# Precision score on test data set.

### YOUR CODE HERE ###
print("Precision Score:",round(precision_score(y_test, y_pred),4))

Precision Score: 0.9641


In [29]:
# Recall score on test data set.

### YOUR CODE HERE ###
print("Recall Score:",round(recall_score(y_test, y_pred),4))

Recall Score: 0.9493


In [30]:
# Accuracy score on test data set.

### YOUR CODE HERE ###
print("Accuracy Score:",round(accuracy_score(y_test, y_pred),4))

Accuracy Score: 0.9529


In [31]:
# F1 score on test data set.

### YOUR CODE HERE ###
print("F1 Score:",round(f1_score(y_test, y_pred),4))

F1 Score: 0.9566


**Question:** How does this model perform based on the four scores?

The model excells in all 4 metrics, this is an indication that the features chosen and the model itself are capable of correctly classifyin data.

### Evaluate the model

Finally, create a table of results that you can use to evaluate the performace of your model.

In [40]:
# Create table of results.

### YOUR CODE HERE ###

table = pd.DataFrame()

table = table.append({'Model':'Tuned Decision Tree',
             'F1':0.943673,
             'Recall':0.935216,
             'Precision':0.952305,
             'Accuracy':0.938887},
            ignore_index=True,)

table = table.append({"Model":"Random Forest",
             "F1":round(f1_score(y_test, y_pred),6),
             "Recall":round(recall_score(y_test, y_pred),6),
             "Precision":round(precision_score(y_test, y_pred),6),
             "Accuracy":round(accuracy_score(y_test, y_pred),6)},
            ignore_index=True)

table

,Model,F1,Recall,Precision,Accuracy
0,Tuned Decision Tree,0.943673,0.935216,0.952305,0.938887
1,Random Forest,0.956640,0.949326,0.964069,0.952891



<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Build a table to compare the performance of the models. Create a DataFrame and use the `append()` function to add the results of each model as a new row.

</details>

**Question:** How does the random forest model compare to the decision tree model you built in the previous lab?

The Random Forest built in this activity performs better than the tuned decision tree from the previous lab in all 4 metrics. This is indication of the RF being a better model for this specific problem.



## **Considerations**


**What are the key takeaways from this lab? Consider important steps when building a model, most effective approaches and tools, and overall results.**

Data cleaning and reduction protocols must always be done prior to any machine learning method is used.

The validation set within the training set is needed in order to reduce the overfitting nature of decision trees, and also reduce the amount of bias in our model.

A decision tree model render more effective results at the cost of more computational power.


**What summary would you provide to stakeholders?**

The random forest model can predict customer satisfaction to an accuracy of 95.2%, precision of 96.4% and a recall of 94.9% meaning that its unlikely the model will mislabel costumers.

The random forest outperformed the decision tree with tuned hyper-parameters in all 4 metrics, thus indicating that the decision tree is a better model altogether for this problem.

### References

[What is the Difference Between Test and Validation Datasets?,  Jason Brownlee](https://machinelearningmastery.com/difference-test-validation-datasets/)

[Decision Trees and Random Forests Neil Liberman](https://towardsdatascience.com/decision-trees-and-random-forests-df0c3123f991)

**Congratulations!** You've completed this lab. However, you may not notice a green check mark next to this item on Coursera's platform. Please continue your progress regardless of the check mark. Just click on the "save" icon at the top of this notebook to ensure your work has been logged